In [22]:
##########################################
#Examples for different configurations:
#
# Assume example data set:
# - Patient1
#   - pat1_TP1_MR
#   - pat1_TP2_MR
#   - pat1_TP1_CT
# - Patient2
#   - pat2_TP1_MR1
#   - pat2_TP1_MR2
#   - pat2_TP2_MR1
#   - pat2_TP1_CT
#   - pat2_TP2_CT

In [23]:
###############################################################################
# Imports
###############################################################################
import os

import avid.common.workflow as workflow
import avid.common.artefact.defaultProps as artefactProps

from avid.selectors import ActionTagSelector as ATS
from avid.actions.pythonAction import PythonNaryBatchAction as naryPythonAction, PythonNaryBatchActionV2

from avid.linkers import CaseLinker, TimePointLinker, FractionLinker
from avid.splitter import SingleSplitter, CaseSplitter, KeyValueSplitter

In [24]:
###############################################################################
# Define ActionTagSelectors
###############################################################################
mr_image_selector = ATS('MR')
ct_image_selector = ATS('CT')
mask_selector = ATS('Seg')

In [25]:
###############################################################################
# Define callable
###############################################################################
def my_function(outputs, **kwargs):
    '''
        Simple print-callable that outputs the combinations of input artifacts.
    '''
    keys = kwargs.keys()
    print_output = 'output: {'
    
    for i, key in enumerate(keys):
        if i != 0:
            print_output += ', '
        print_output += '{}:{}'.format(key, [os.path.basename(mr) for mr in kwargs[key]])
    print_output += '}'
    
    print(print_output)
    with open(outputs[0], "w") as ofile:
        ofile.write(str(kwargs))

In [26]:
###############################################################################
# Initialize session
###############################################################################
session =  workflow.initSession(bootstrapArtefacts=os.path.join(os.getcwd(),'output', 'example.avid'),
                                sessionPath=os.path.join(os.getcwd(),'output', 'example'),
                                expandPaths=True,
                                debug=True)

In [27]:
###############################################################################
# Example 1: only select a specific modality (here MR) and make a call per image
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector, 
                        actionTag="example_1", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do()

2021-12-08 20:26:14,334 [INFO] Starting action: PythonNaryBatchActionV2_example_1 (UID: 62b8ee06-47d9-44df-8c23-08107745f8d1) ...
2021-12-08 20:26:14,336 [INFO] Starting action: my_function (UID: 09fc57d9-35c5-4239-9de5-02996007c0af) ...
output: {primaryInput:['pat1_TP1_MR.txt']}
2021-12-08 20:26:14,340 [INFO] Finished action: my_function (UID: 09fc57d9-35c5-4239-9de5-02996007c0af) -> SUCCESS
2021-12-08 20:26:14,341 [INFO] Starting action: my_function (UID: 25b2c53e-ff9a-43a2-ab10-5408beb1f2d0) ...
output: {primaryInput:['pat1_TP2_MR.txt']}
2021-12-08 20:26:14,345 [INFO] Finished action: my_function (UID: 25b2c53e-ff9a-43a2-ab10-5408beb1f2d0) -> SUCCESS
2021-12-08 20:26:14,345 [INFO] Starting action: my_function (UID: bac4c85b-3013-4151-81dc-279fbcf0aacd) ...
output: {primaryInput:['pat2_TP1_MR1.txt']}
2021-12-08 20:26:14,350 [INFO] Finished action: my_function (UID: bac4c85b-3013-4151-81dc-279fbcf0aacd) -> SUCCESS
2021-12-08 20:26:14,351 [INFO] Starting action: my_function (UID: b8970

In [28]:
###############################################################################
#Example 2: like example 1 but choose another variable name
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        actionTag="example_2", 
                        generateCallable=my_function, 
                        passOnlyURLs=True).do()

2021-12-08 20:26:14,382 [INFO] Starting action: PythonNaryBatchActionV2_example_2 (UID: 59dc4f73-4a77-4260-b3da-2d2fec467a20) ...
2021-12-08 20:26:14,384 [INFO] Starting action: my_function (UID: 3a3ddb4e-02bf-4372-af80-5f34b1761c97) ...
output: {mr_images:['pat1_TP1_MR.txt']}
2021-12-08 20:26:14,388 [INFO] Finished action: my_function (UID: 3a3ddb4e-02bf-4372-af80-5f34b1761c97) -> SUCCESS
2021-12-08 20:26:14,389 [INFO] Starting action: my_function (UID: 3e3bc0a8-c30c-4f5d-bf75-ffb6bb828147) ...
output: {mr_images:['pat1_TP2_MR.txt']}
2021-12-08 20:26:14,394 [INFO] Finished action: my_function (UID: 3e3bc0a8-c30c-4f5d-bf75-ffb6bb828147) -> SUCCESS
2021-12-08 20:26:14,394 [INFO] Starting action: my_function (UID: a3c213fc-dba2-4f66-b219-3620d67c9a39) ...
output: {mr_images:['pat2_TP1_MR1.txt']}
2021-12-08 20:26:14,398 [INFO] Finished action: my_function (UID: a3c213fc-dba2-4f66-b219-3620d67c9a39) -> SUCCESS
2021-12-08 20:26:14,399 [INFO] Starting action: my_function (UID: dfa09e2a-7e3f-

In [35]:
###############################################################################
#Example 3: select MR and CT images and call them pairwise (folding both sets per case)
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors = {'ct_images': ct_image_selector},
                        actionTag="example_3",
                        generateCallable=my_function,
                        passOnlyURLs=True).do().tagSelector

2021-12-08 20:32:02,256 [INFO] Starting action: PythonNaryBatchActionV2_example_3 (UID: cdc62e99-0025-4231-96c8-de90afe7dc37) ...
2021-12-08 20:32:02,264 [INFO] Starting action: my_function (UID: 8b81250c-214b-433a-a9fa-f3550894adae) ...
output: {mr_images:['pat1_TP1_MR.txt'], ct_images:['pat1_TP1_CT.txt']}
2021-12-08 20:32:02,269 [INFO] Finished action: my_function (UID: 8b81250c-214b-433a-a9fa-f3550894adae) -> SUCCESS
2021-12-08 20:32:02,270 [INFO] Starting action: my_function (UID: a285b877-04d3-49c9-a94e-02a25349fb1a) ...
output: {mr_images:['pat1_TP2_MR.txt'], ct_images:['pat1_TP1_CT.txt']}
2021-12-08 20:32:02,275 [INFO] Finished action: my_function (UID: a285b877-04d3-49c9-a94e-02a25349fb1a) -> SUCCESS
2021-12-08 20:32:02,276 [INFO] Starting action: my_function (UID: f5864d44-a4b2-4221-a630-fd52a41985d6) ...
output: {mr_images:['pat2_TP1_MR1.txt'], ct_images:['pat2_TP1_CT.txt']}
2021-12-08 20:32:02,281 [INFO] Finished action: my_function (UID: f5864d44-a4b2-4221-a630-fd52a41985d6

In [36]:
###############################################################################
#Example 3b:  select MR and CT images and call them pairwise (but only MRs and CTs of same patient and timepoint)
###############################################################################
with session:
    splitProperties=[artefactProps.CASE]
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                            primaryAlias='mr_images',
                            additionalInputSelectors = {'additional': ct_image_selector},
                            linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                            actionTag="example_3b",
                            generateCallable=my_function,
                            passOnlyURLs=True).do().tagSelector

2021-12-08 20:35:20,471 [INFO] Starting action: PythonNaryBatchActionV2_example_3b (UID: 46ccf9a0-67ae-47bf-969e-4fd6862a7cff) ...
2021-12-08 20:35:20,475 [INFO] Starting action: my_function (UID: 5394ad68-d304-491f-9e09-4ce328c37652) ...
output: {mr_images:['pat1_TP1_MR.txt'], additional:['pat1_TP1_CT.txt']}
2021-12-08 20:35:20,480 [INFO] Finished action: my_function (UID: 5394ad68-d304-491f-9e09-4ce328c37652) -> SUCCESS
2021-12-08 20:35:20,482 [INFO] Starting action: my_function (UID: cd6686a9-18cf-4ee3-8518-88e5009f2377) ...
output: {mr_images:['pat2_TP1_MR1.txt'], additional:['pat2_TP1_CT.txt']}
2021-12-08 20:35:20,487 [INFO] Finished action: my_function (UID: cd6686a9-18cf-4ee3-8518-88e5009f2377) -> SUCCESS
2021-12-08 20:35:20,488 [INFO] Starting action: my_function (UID: fe1e3119-b662-4a39-afb6-55c8377bd397) ...
output: {mr_images:['pat2_TP1_MR2.txt'], additional:['pat2_TP1_CT.txt']}
2021-12-08 20:35:20,493 [INFO] Finished action: my_function (UID: fe1e3119-b662-4a39-afb6-55c8377

In [37]:
###############################################################################
#Example 3c:  select MR and CT images and call them blockwise (but only MRs and CTs of same patient)
###############################################################################
with session:
    splitProperties=[artefactProps.CASE]
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors = {'additional': ct_image_selector},
                        linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                        splitter={'mr_images': KeyValueSplitter(*splitProperties), 'additional': CaseSplitter()},
                        actionTag="example_3b", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do().tagSelector

2021-12-08 20:37:13,009 [INFO] Starting action: PythonNaryBatchActionV2_example_3b (UID: 0475fc69-cf7e-442f-b0f2-64fbeb0bf43c) ...
2021-12-08 20:37:13,011 [INFO] Starting action: my_function (UID: deb0ca36-20b3-4d1c-93fe-04545ca5145c) ...
output: {mr_images:['pat1_TP1_MR.txt', 'pat1_TP2_MR.txt'], additional:['pat1_TP1_CT.txt']}
2021-12-08 20:37:13,017 [INFO] Finished action: my_function (UID: deb0ca36-20b3-4d1c-93fe-04545ca5145c) -> SUCCESS
2021-12-08 20:37:13,018 [INFO] Starting action: my_function (UID: edcc0983-30c0-4cd6-a320-a5940b9bd4f9) ...
output: {mr_images:['pat2_TP1_MR1.txt', 'pat2_TP1_MR2.txt', 'pat2_TP2_MR1.txt'], additional:['pat2_TP1_CT.txt', 'pat2_TP2_CT.txt']}
2021-12-08 20:37:13,024 [INFO] Finished action: my_function (UID: edcc0983-30c0-4cd6-a320-a5940b9bd4f9) -> SUCCESS
2021-12-08 20:37:13,026 [INFO] Finished action: PythonNaryBatchActionV2_example_3b (UID: 0475fc69-cf7e-442f-b0f2-64fbeb0bf43c) -> SUCCESS
2021-12-08 20:37:13,026 [INFO] Successful actions: 5.
2021-12-

In [31]:
###############################################################################
#Example 4: select MR images and masks and call them casewise
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors ={'additional': mask_selector},
                        linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                        splitter={'mr_images': CaseSplitter(), 'additional': CaseSplitter()},
                        actionTag="example_4", 
                        generateCallable=my_function, 
                        passOnlyURLs=True).do().tagSelector

2021-12-08 20:26:14,558 [INFO] Starting action: PythonNaryBatchActionV2_example_4 (UID: 2560162e-650a-4a63-9e08-d9bd415c822c) ...
2021-12-08 20:26:14,560 [INFO] Starting action: my_function (UID: 1f8937f4-39c9-4b52-92b4-8e86f1a1968e) ...
output: {mr_images:['pat1_TP1_MR.txt', 'pat1_TP2_MR.txt'], additional:['pat1_TP1_Seg1.txt', 'pat1_TP1_Seg2.txt', 'pat1_TP2_Seg1.txt']}
2021-12-08 20:26:14,565 [INFO] Finished action: my_function (UID: 1f8937f4-39c9-4b52-92b4-8e86f1a1968e) -> SUCCESS
2021-12-08 20:26:14,566 [INFO] Starting action: my_function (UID: 4cd0d2a8-ed6b-4081-bf0b-13456bdfdabf) ...
output: {mr_images:['pat2_TP1_MR1.txt', 'pat2_TP1_MR2.txt', 'pat2_TP2_MR1.txt'], additional:['pat2_TP1_Seg1.txt', 'pat2_TP2_Seg1.txt']}
2021-12-08 20:26:14,570 [INFO] Finished action: my_function (UID: 4cd0d2a8-ed6b-4081-bf0b-13456bdfdabf) -> SUCCESS
2021-12-08 20:26:14,571 [INFO] Starting action: my_function (UID: 48dabd68-36ae-455a-8786-e6c2ecc2a0e5) ...
output: {mr_images:['pat3_TP1_MR.txt'], addit